In [4]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
full_dev_data  = '/home/ashrafs/projects/dragon/data/fever1/old-format/enriched_feverous_dev.jsonl'

In [32]:
dev_fever_data = '/home/ashrafs/projects/dragon/data/fever/Main/dev-split.jsonl'
train_fever_data = '/home/ashrafs/projects/dragon/data/fever/Main/train-split.jsonl'
test_fever_data = '/home/ashrafs/projects/dragon/data/fever/Main/test-split.jsonl'


In [6]:
dev_fever_output = '/home/ashrafs/projects/dragon/data/fever/Main/dev.jsonl'
train_fever_output = '/home/ashrafs/projects/dragon/data/fever/Main/train.jsonl'
test_fever_output = '/home/ashrafs/projects/dragon/data/fever/Main/test.jsonl'



In [5]:
def load_data(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data



In [35]:
# Load the data

dev_data = load_data(dev_fever_data)
train_data = load_data(train_fever_data)
test_data = load_data(test_fever_data)

# Now, data_list contains all the data from the JSONL file
# Count the claims
number_of_claims = len(dev_data)
number_of_claims


10

In [8]:
for data in dev_data:
    print(data.get("CLAIM"))
    print("LABEL: ", data.get("LABEL"), " ID: ", data.get("id"))
    print("----------")
    

NameError: name 'dev_data' is not defined

In [7]:
def convert_to_fever_format(data_list):
    fever_format_data = []
    label_to_key = {"SUPPORTS": "A", "REFUTES": "B", "NOT ENOUGH INFO": "C"}
    placeholder_text = "which:"

    for data in data_list:
        claim_id = data.get("id")
        label = data.get("LABEL")

        fever_item = {
            "id": claim_id,
            "question": {
                "stem": f"{data.get('CLAIM')}, {placeholder_text}",
                "choices": [
                    {"text": "SUPPORTS", "label": "A"},
                    {"text": "REFUTES", "label": "B"},
                    {"text": "NOT ENOUGH INFO", "label": "C"},
                ]
            },
            "answerKey": label_to_key.get(label)  # Default to "C" if label is not found
        }

        fever_format_data.append(fever_item)

    return fever_format_data



In [39]:
# Assuming `data_list` is your list of data
converted_dev= convert_to_fever_format(dev_data)
converted_train= convert_to_fever_format(train_data)
converted_test= convert_to_fever_format(test_data)

# Example of printing the first item in the converted format
print(converted_dev[0])


{'id': 21641, 'question': {'stem': 'Abraham Annan Adjei has never played professional football., which:', 'choices': [{'text': 'SUPPORTS', 'label': 'A'}, {'text': 'REFUTES', 'label': 'B'}, {'text': 'NOT ENOUGH INFO', 'label': 'C'}]}, 'answerKey': 'B'}


In [13]:
def save_data_as_jsonl(data_subset, jsonl_file_path):
    with open(jsonl_file_path, 'w', encoding='utf-8') as file:
        for item in data_subset:
            json.dump(item, file)
            file.write('\n')  # Write a newline character after each JSON object




In [30]:
# Save the data subsets
save_data_as_jsonl(converted_dev, dev_fever_output)
save_data_as_jsonl(converted_train, train_fever_output)
save_data_as_jsonl(converted_test, test_fever_output)

In [10]:
#converting full dev set into format of MCQ:
# Load the data

dev_data = load_data(full_dev_data)

# Now, data_list contains all the data from the JSONL file
# Count the claims
number_of_claims = len(dev_data)
print("No of Claims: ", number_of_claims)
converted_dev= convert_to_fever_format(dev_data)
print(converted_dev[0])


No of Claims:  7890
{'id': 7389, 'question': {'stem': 'Algebraic logic has five Logical system and Lindenbaum–Tarski algebra which includes Physics algebra and Nodal algebra (provide models of propositional modal logics)., which:', 'choices': [{'text': 'SUPPORTS', 'label': 'A'}, {'text': 'REFUTES', 'label': 'B'}, {'text': 'NOT ENOUGH INFO', 'label': 'C'}]}, 'answerKey': 'B'}


In [14]:
len(converted_dev)

7890

In [18]:
def split_and_save_data(data_list):
    
    train_data, temp_data = train_test_split(data_list, test_size=0.2, random_state=42)
    dev_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

    # Save each subset to its corresponding jsonl file
    save_data_as_jsonl(train_data, train_fever_output)
    save_data_as_jsonl(dev_data, dev_fever_output)
    save_data_as_jsonl(test_data, test_fever_output)

    return len(train_data), len(dev_data), len(test_data)


total_data_count = len(converted_dev)  
train_count, dev_count, test_count = split_and_save_data(converted_dev)

print(f"Data split into train: {train_count} (approx. {train_count/total_data_count*100:.2f}%), "
      f"dev: {dev_count} (approx. {dev_count/total_data_count*100:.2f}%), "
      f"test: {test_count} (approx. {test_count/total_data_count*100:.2f}%)")

Data split into train: 6312 (approx. 80.00%), dev: 789 (approx. 10.00%), test: 789 (approx. 10.00%)
